In [1]:
import numpy as np

from system import (
    mdp,
    reward,
    states,
    num_states,
    num_actions,
    to_idx,
    get_valid_actions,
)
from model import Model
from policy import Policy, PolicyInit

In [7]:
qarr = np.random.uniform(0, 2, (num_states, num_actions))
policy = Policy(num_states, num_actions, PolicyInit.RANDOM)
prob = np.ones(num_states*num_actions)/(num_states*num_actions)
possi = []
for s in range(num_states):
    for a in range(num_actions):
        possi.append([s, a])
possi = np.array(possi)
returns = [[[] for a in range(num_actions)]for s in range(num_states)]

In [ ]:
i = 0
T = 100
while i < 1000:
    ipair = np.random.choice(possi, p = prob)
    fo = {}
    for j in range(T):
        s = estates[j]
        a = eactions[j]
        r = erewards[j]
        if [s, a] not in fo:
            fo[[s, a]] = j
    g = 0
    for j in range(T):
        s = estates[T - j - 1]
        a = eactions[T - j - 1]
        r = erewards[T - j - 1]
        g = gamma*g + r
        if T - j - 1 == fo[[s, a]]:
            returns[s][a].append(g)
            qarr[s, a] = np.mean(returns[s][a])
    ua = np.argmax(qarr, axis = 1)
    for s in range(num_states):
        policy.set_action(s, ua[s])